In [37]:
import torch
import torch.nn as nn
import matplotlib as plt
from tqdm.auto import tqdm

from torch.autograd import Variable

from data import iris

In [38]:
!head data/iris.data.txt

sepal_length_cm,sepal_width_cm,petal_length_cm,petal_width_cm,class
5.1,3.5,1.4,0.2,Iris-setosa
4.9,3.0,1.4,0.2,Iris-setosa
4.7,3.2,1.3,0.2,Iris-setosa
4.6,3.1,1.5,0.2,Iris-setosa
5.0,3.6,1.4,0.2,Iris-setosa
5.4,3.9,1.7,0.4,Iris-setosa
4.6,3.4,1.4,0.3,Iris-setosa
5.0,3.4,1.5,0.2,Iris-setosa
4.4,2.9,1.4,0.2,Iris-setosa


In [39]:
class IrisNet(nn.Module):

    def __init__(self, input_size, hidden_size, num_classes):
        super(IrisNet, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(input_size, hidden_size),
            nn.ReLU(inplace=True),
            nn.Linear(hidden_size, num_classes)
        )

    def forward(self, x):
        return self.net(x)

In [40]:
batch_size = 4
iris_csv = "./data/iris.data.txt"

train, test = iris.get_datasets(iris_csv)

train_loader = torch.utils.data.DataLoader(train, batch_size = batch_size, shuffle = True)
test_loader = torch.utils.data.DataLoader(test, batch_size = batch_size, shuffle = True)

In [41]:
criterion = nn.CrossEntropyLoss()
n_epochs = 20
device = 'cuda'

iris_model = IrisNet(4, 50, 3).to(device)
iris_opt = torch.optim.SGD(iris_model.parameters(), lr=0.001)

In [42]:
def get_loss(iris_model, criterion, input, ground_truth):
    iris_pred = iris_model(input)
    return criterion(iris_pred, ground_truth)


In [43]:
curr_step = 0
mean_loss = 0
for epoch in range(n_epochs):
    for flower_batch, classes in tqdm(train_loader):
        flower_batch = Variable(flower_batch).to(device)
        classes = Variable(classes).to(device)

        iris_model.train()

        iris_opt.zero_grad()

        iris_loss = get_loss(iris_model, criterion, flower_batch, classes)
        
        iris_loss.backward(retain_graph = True)

        iris_opt.step()

